In [ ]:
import pandas as pd, glob, numpy as np
from datetime import date, timedelta

In [ ]:
# Daily processing
daily_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Stations\Climate_Sensitive_Stations-GRDC\2025-02-13_17-18_Daily"
monthly_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Stations\Climate_Sensitive_Stations-GRDC\2025-02-13_17-18_Monthly"
watersheds_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Sts_Drainage_Areas\GRDC_Watersheds"
ext = "*.txt"

# TerraClimate available period
terra_st_yr = 1958
terra_ed_yr = 2023
sdate = date(terra_st_yr, 1, 1) # start date
edate = date(terra_ed_yr, 12, 31) # end date

In [ ]:
period_dates = [(sdate + timedelta(days=i)).strftime('%Y-%m-%d') for i in range((edate - sdate).days + 1)]

# Create a DF whose index corresponds to the TerraClimate dates
daily_df_sts = pd.DataFrame({"YYYY-MM-DD": period_dates}).set_index("YYYY-MM-DD")
daily_df_sts

In [ ]:
# Daily processing
archivos = glob.glob(daily_folder + "//" + ext)

for archivo in archivos:

    sts_dict = {}
    file_name = archivo.split("\\")[-1]
    print("Reading file: " + file_name)

    id_station = file_name.split("_")[0]
    data_matrix = []

    # Specify encoding explicitly
    with open(archivo, 'r', encoding='ISO-8859-1') as inFile:    
        data_matrix = inFile.readlines()[37:] # starting data line in the file

    date_array = []
    value_array = []

    for data in data_matrix:
        line = data.split(";")
        line_date = line[0]

        try:
            line_value = float(line[-1])
        except ValueError:
            print(f"Skipping invalid value in {archivo}: {line[-1]}")
            continue

        if int(line_value) == -999: # This avoids saving no-data values (-999)
            line_value = np.nan

        date_array.append(line_date)
        value_array.append(line_value)

    if len(value_array) == 0: # This avoids saving files with no station data
        print(f"Skipping station {id_station} due to no data")
        continue

    sts_dict["YYYY-MM-DD"] = date_array
    sts_dict[id_station] = value_array

    # Create a DF whose index corresponds to the TerraClimate dates
    temp_df = pd.DataFrame(sts_dict).set_index("YYYY-MM-DD")
    daily_df_sts = daily_df_sts.join(temp_df) # left join on index

# Drop columns with all NaN values
daily_df_cleaned = daily_df_sts.dropna(axis=1, how='all')
#daily_df_cleaned.to_csv(daily_folder + "\_DataFrames\Joined_Daily_Sts_DFs.csv")
daily_df_cleaned

In [ ]:
years = range(terra_st_yr, terra_ed_yr + 1)
months = range(1, 12 + 1)
yr_mth= [str(a)+ "-" + str(b).zfill(2) for a in years for b in months]

# Create a DF whose index corresponds to the TerraClimate monthly dates
monthly_df_sts = pd.DataFrame({"YYYY-MM": yr_mth}).set_index("YYYY-MM")
monthly_df_sts

In [ ]:
# Read the CSV file with UTF-8 encoding that contains information on the CSS-related watersheds. This file contains only 1,236 records as 9 CSS stations did not have delineated watersheds provided by GRDC
drain_areas_df = pd.read_csv(watersheds_folder + "\\CSS-GRDC_Watersheds.csv") # grdc_no == station_no, area == CATCHMENT_SIZE 
drain_areas_df

In [ ]:
# This ensures working only with stations whose watersheds are bigger than three TerraClimate pixels, considering a pixel size of 4 km. Therefore, 4 x 4 x 3 = 48
# This facilitates zonal statistics, which are executed after the water balance to extract values from the resulting surfaces for the ultimate watersheds. 
filtered_drain_areas_df = drain_areas_df[drain_areas_df["area"] > 48]
filtered_drain_areas_df

In [ ]:
sts_ids = filtered_drain_areas_df["grdc_no"].astype(int).to_list()
print(sts_ids)

In [ ]:
# Monthly processing
archivos = glob.glob(monthly_folder + "//" + ext)

for archivo in archivos:

    sts_dict = {}
    file_name = archivo.split("\\")[-1]
    print("Reading file: " + file_name)

    id_station = file_name.split("_")[0]

    if int(id_station) not in sts_ids: # This avoids reading files of stations that do not have their respective watersheds
        continue

    data_matrix = []

    # Specify encoding explicitly
    with open(archivo, 'r', encoding='ISO-8859-1') as inFile:    
        data_matrix = inFile.readlines()[39:] # starting data line in the file

    date_array = []
    value_array = []

    for data in data_matrix:
        line = data.split(";")
        line_date = '-'.join(line[0].split("-")[0:2])

        try:
            line_value = float(line[-2])
        except ValueError:
            print(f"Skipping invalid value in {archivo}: {line[-2]}")
            continue

        if int(line_value) == -999: # This avoids saving no-data values (-999)
            line_value = np.nan

        date_array.append(line_date)
        value_array.append(line_value)

    if len(value_array) == 0: # This avoids saving files with no station data
        print(f"Skipping station {id_station} due to no data")
        continue

    sts_dict["YYYY-MM"] = date_array
    sts_dict[id_station] = value_array

    # Create a DF whose index corresponds to the TerraClimate monthly dates
    temp_df = pd.DataFrame(sts_dict).set_index("YYYY-MM")
    monthly_df_sts = monthly_df_sts.join(temp_df) # left join on index

# Drop columns with all NaN values
monthly_df_cleaned = monthly_df_sts.dropna(axis=1, how='all')
#monthly_df_cleaned.to_csv(daily_folder + "\_DataFrames\Joined_Monthly_Sts_DFs.csv")
monthly_df_cleaned

To identify stations with at least 30 complete water-years (Oct-Sep) in a DataFrame where:

- The index is in YYYY-MM format.
- The columns are stations with their monthly multiannual values.

1. Convert the index to a DateTime format
Since the index is in YYYY-MM format, convert it to a proper datetime format for easier filtering and resampling.

In [ ]:
reindexed_monthly_df_cleaned = monthly_df_cleaned.copy()
reindexed_monthly_df_cleaned.index = pd.to_datetime(reindexed_monthly_df_cleaned.index, format='%Y-%m')
reindexed_monthly_df_cleaned

2. Define Water Years (Oct-Sep)
The water year starts in October and ends in September of the following year. You can define a water-year label as the year of the September within that water year.

In [ ]:
reindexed_monthly_df_cleaned['water_year'] = reindexed_monthly_df_cleaned.index.to_series().apply(lambda x: x.year if x.month < 10 else x.year + 1)
reindexed_monthly_df_cleaned

3. Count Complete Water Years for Each Station
Now, we group by water year and count the number of non-null monthly values per station. A complete water year must have 12 valid values for a given station.

- This method ensures that:

    - Only stations with 30+ water years (each with all 12 months) are selected.
    
    - The approach is flexible for datasets with missing months.

In [ ]:
# Count valid months per water year per station
valid_months_per_wy = reindexed_monthly_df_cleaned.groupby('water_year').count()

# Identify stations with at least 30 complete water years. This guarantees that we are considering the same approach of TerraClimate authors for results validation
stations_with_30_wy = (valid_months_per_wy == 12).sum(axis=0) >= 30
selected_stations = stations_with_30_wy[stations_with_30_wy].index.tolist()
print(selected_stations)  # List of stations with at least 30 complete water years
print("The final amount of stations with at least 30 complete water years is", len(selected_stations))

In [ ]:
# Export the final dataframes with the stations to be considered
final_daily_df = daily_df_cleaned[selected_stations]
final_daily_df.to_csv(daily_folder + "\_DataFrames\Joined_Daily_Sts_DFs.csv")

final_monthly_df = monthly_df_cleaned[selected_stations]
final_monthly_df.to_csv(monthly_folder + "\_DataFrames\Joined_Monthly_Sts_DFs.csv")

In [ ]:
final_daily_df = pd.read_csv_csv(daily_folder + "\_DataFrames\Joined_Daily_Sts_DFs.csv", index_col="YYYY-MM")
final_daily_df

In [ ]:
final_monthly_df = pd.read_csv_csv(monthly_folder + "\_DataFrames\Joined_Monthly_Sts_DFs.csv", index_col="YYYY-MM")
final_monthly_df